# Utilisation de SQLite dans Python 3

## 0. Remarque préliminaire

J'utilise Jupyter pour la présentation, mais vous pouvez utiliser l'interface que vous voulez avec Python 3.

## 1. Connexion et curseur

In [1]:
import sqlite3
conn = sqlite3.connect('emp.db')
c = conn.cursor()

c est un curseur, on va l'utiliser pour nos requêtes.

In [2]:
conn.commit()

Sauvegarder (`commit`) les changements

In [3]:
conn.close()

Fermer la connexion. Si des changements ont été faits sans `commit`, ils sont perdus.

## 2. Requêtes de base

In [4]:
import sqlite3
conn = sqlite3.connect('emp.db')
c = conn.cursor()

In [5]:
c

In [6]:
c.execute('SELECT * FROM dept')

Renvoie un curseur, on va pouvoir aller chercher les résultats grâce à lui.

In [7]:
c.fetchone()

(10, 'finance', 'paris')

In [8]:
c.fetchone()

(20, 'recherche', 'nice')

`c.fetchone()` renvoie une ligne, sous forme d'uplet (au sens de Python)

`c.fetchall()` renvoie toutes les lignes qui restent, sous forme de liste d'uplets

In [9]:
c.fetchall()

[(30, 'vente', 'lyon'), (40, 'fabrication', 'marseille')]

On peut aussi utiliser les guillemets triples pour écrire une requête sur plusieurs lignes.

In [10]:
c.execute('''SELECT *
             FROM emp, dept
             WHERE emp.numd = dept.numd''')
c.fetchone()

(1090,
 'Leroy',
 'president',
 None,
 '2017-01-01',
 5000,
 None,
 10,
 10,
 'finance',
 'paris')

On peut utiliser la syntaxe de Python sur les listes pour afficher toutes les lignes produites.

In [11]:
c.execute('SELECT * FROM emp, dept WHERE emp.numd = dept.numd')
for ligne in c.fetchall():
    print(ligne)

(1090, 'Leroy', 'president', None, '2017-01-01', 5000, None, 10, 10, 'finance', 'paris')
(1040, 'Mercier', 'directeur', 1090, '2017-03-01', 4000, None, 20, 20, 'recherche', 'nice')
(1060, 'Noiret', 'directeur', 1090, '2017-01-20', 3500, None, 30, 30, 'vente', 'lyon')
(1070, 'Lesage', 'directeur', 1090, '2017-06-10', 3400, None, 10, 10, 'finance', 'paris')
(1080, 'Dubois', 'ingenieur', 1040, '2017-07-20', 4000, None, 20, 20, 'recherche', 'nice')
(2030, 'Chatel', 'ingenieur', 1040, '2017-08-20', 3000, None, 20, 20, 'recherche', 'nice')
(2020, 'Fremont', 'secretaire', 1060, '2018-01-20', 1100, None, 30, 30, 'vente', 'lyon')
(1020, 'Biraud', 'commercial', 1060, '2017-02-10', 2000, 400, 30, 30, 'vente', 'lyon')
(1030, 'Berger', 'commercial', 1060, '2017-02-01', 1200, 700, 30, 30, 'vente', 'lyon')
(1050, 'Martin', 'commercial', 1060, '2017-03-10', 1300, 1600, 30, 30, 'vente', 'lyon')
(2000, 'Benain', 'commercial', 1060, '2017-02-20', 2000, 2100, 30, 30, 'vente', 'lyon')
(2040, 'Villard', 'se

On a ainsi itéré sur la liste obtenue par `c.fetchall()`, mais on peut aussi traiter le curseur `c` comme un itérateur.

In [12]:
for ligne in c.execute('SELECT * FROM projet'):
    print(ligne)
    

(101, 'Projet stade')
(102, 'Projet centre commercial')
(103, 'Projet tramway')
(105, 'projet génial')
(106, 'augmenter son salaire')
(107, 'projet génial')
(108, 'augmenter son salaire')


## 3. Utilisation de variables Python dans les requêtes

Fréquemment, les opérations SQL vont utiliser des variables Python.<br/>
L'exemple suivant est un mauvais exemple, à ne pas suivre.

In [13]:
nom_projet = 'projet génial'
c.executescript("INSERT INTO projet (nomp) VALUES  ('{}');".format(nom_projet))

La méthode `c.executescript(string)` permet d'exécuter plusieurs commandes SQL, mais cette technique est dangereuse.
En effet, procéder ainsi est peu sûr et rend votre programme vulnérable à une attaque par injection de SQL : cela consiste à ajouter une requête différente dans ce qui devrait être une chaîne de caractères banale.

### Injection de SQL

In [14]:
c.execute('SELECT * FROM emp WHERE matr = 2040;')
c.fetchall()

[(2040, 'Villard', 'secretaire', 1070, '2017-08-01', 1800, None, 10)]

Supposons que la variable `poste` soit obtenue à partir d'une exécution non sécurisée, par exemple, directement obtenue depuis un formulaire web. Alors une personne mal intentionnée a réussi à faire en sorte que

In [15]:
nom_projet = 'augmenter son salaire"); UPDATE emp SET salaire = salaire + 200 WHERE matr = ("2040'

Alors, le script exécuté par SQL sera

In [16]:
print('INSERT INTO projet (nomp) VALUES  ("{}");'.format(nom_projet))

INSERT INTO projet (nomp) VALUES  ("augmenter son salaire"); UPDATE emp SET salaire = salaire + 200 WHERE matr = ("2040");


Observons son effet

In [17]:
c.executescript('INSERT INTO projet (nomp) VALUES  ("{}");'.format(nom_projet))

In [18]:
c.execute('SELECT * FROM emp WHERE matr = 2040;')
c.fetchall()

[(2040, 'Villard', 'secretaire', 1070, '2017-08-01', 2000, None, 10)]

Ainsi, la personne de matricule 2040 a augmenté son salaire de 200€.

À la place, utilisez `c.execute(str)` qui exécute au plus une requête et les espaces réservés prévus par cette méthode.

In [19]:
u = ('écrire du sql raisonnable',)
c.execute('INSERT INTO projet (nomp) VALUES (?)',u)

Le point d'interrogation `?` peut être utilisé comme espace réservé pour n'importe quelle valeur, puis fournissez un uplet de valeurs comme second argument de la méthode `execute` appliquée au curseur `c`.

In [20]:
nouveaux = [(4000, 'Dubois', 'secretaire', 1060, '2019-11-26', 1300, 0, 20),
             (4100, 'Durand', 'commercial', 1060, '2019-11-26', 2000, 1000, 20)
            ]

In [21]:
c.executemany('INSERT INTO emp VALUES (?,?,?,?,?,?,?,?)', nouveaux)

On peut aussi exécuter pour une liste d'uplets avec `executemany`.

In [22]:
c.execute('SELECT * FROM emp')

In [23]:
c.fetchall()

[(1090, 'Leroy', 'president', None, '2017-01-01', 5000, None, 10),
 (1040, 'Mercier', 'directeur', 1090, '2017-03-01', 4000, None, 20),
 (1060, 'Noiret', 'directeur', 1090, '2017-01-20', 3500, None, 30),
 (1070, 'Lesage', 'directeur', 1090, '2017-06-10', 3400, None, 10),
 (1080, 'Dubois', 'ingenieur', 1040, '2017-07-20', 4000, None, 20),
 (2030, 'Chatel', 'ingenieur', 1040, '2017-08-20', 3000, None, 20),
 (2020, 'Fremont', 'secretaire', 1060, '2018-01-20', 1100, None, 30),
 (1020, 'Biraud', 'commercial', 1060, '2017-02-10', 2000, 400, 30),
 (1030, 'Berger', 'commercial', 1060, '2017-02-01', 1200, 700, 30),
 (1050, 'Martin', 'commercial', 1060, '2017-03-10', 1300, 1600, 30),
 (2000, 'Benain', 'commercial', 1060, '2017-02-20', 2000, 2100, 30),
 (2040, 'Villard', 'secretaire', 1070, '2017-08-01', 2000, None, 10),
 (1010, 'Leclerc', 'secretaire', 2030, '2017-01-01', 1000, None, 20),
 (2010, 'Clement', 'secretaire', 1080, '2018-01-10', 1200, None, 20),
 (3000, 'Dubois', 'secretaire', 1060, 

In [24]:
conn.close()